# Programa para validar precipitação acumulada simulada. Validacao feita com dados de pluviometros e MERGE.
## Os dados foram interpolados para uma mesma grade de 0.01 x 0.01o para o calculo dos indices
## Interpolacao dos dados dos pluviometros utilizando interpolacao IDW
## Indices estisticos:
### Viés
### RMSE
### Coeficiente de correlacao de Spearman
#### OBS: Os indices tambem foram calculados utilizando o ponto mais proximo das estacoes (pluviometros)
#### OBS 2: o programa Regrid_BRAMS_MERGE.ipynb mostra a interpolacao feita com os dados simulados e do MERGE

In [1]:
import numpy as np
import pandas as pd
from pysteps import verification
import xarray as xr

Pysteps configuration file found at: /home/mariana/anaconda3/envs/python39/lib/python3.9/site-packages/pysteps/pystepsrc



In [2]:
# Leitura dos dados dos pluvimetros
dados = pd.read_table('filename.txt', sep='\s+')
# Leitura dos dados interpolados da simulacao 
model_interp = xr.open_dataset('filename.nc')
var_model = model_interp['precip'].values
# Leitura dos dados simulados sem interpolacao
model = xr.open_dataset('filename.nc')
precip_model = model['precip'].sel(lat=slice(-23.87, -23.15), lon=slice(-47.08, -46.0))
# Leitura dos dados do MERGE
merge = xr.open_dataset('filename.nc')
var_merge = merge['precip'].values

In [3]:
x = dados['Lon']
y = dados['Lat']
z = dados['Chuva']

In [4]:
# Determinacao da grade para interpolacao
xi = np.arange (-47.08, -45.99, 0.01)
yi = np.arange (-23.87, -23.22, 0.01)

In [5]:
nx, ny = len(xi),len(yi)
nx,ny

(109, 66)

In [6]:
xi,yi = np.meshgrid(xi,yi)
xi, yi = xi.flatten(), yi.flatten()

In [7]:
def distance_matrix(x0, y0, x1, y1):
    """ Make a distance matrix between pairwise observations.
    Note: from <http://stackoverflow.com/questions/1871536> 
    """
    
    obs = np.vstack((x0, y0)).T
    interp = np.vstack((x1, y1)).T

    d0 = np.subtract.outer(obs[:,0], interp[:,0])
    d1 = np.subtract.outer(obs[:,1], interp[:,1])
    
    # calculate hypotenuse
    return np.hypot(d0, d1)

In [8]:
def simple_idw(x, y, z, xi, yi, power=3):
    """ Simple inverse distance weighted (IDW) interpolation 
    Weights are proportional to the inverse of the distance, so as the distance
    increases, the weights decrease rapidly.
    The rate at which the weights decrease is dependent on the value of power.
    As power increases, the weights for distant points decrease rapidly.
    """
    
    dist = distance_matrix(x,y, xi,yi)

    # In IDW, weights are 1 / distance
    weights = 1.0/(dist+1e-12)**power

    # Make weights sum to one
    weights /= weights.sum(axis=0)

    # Multiply the weights for each interpolated point by all observed Z-values
    return np.dot(weights.T, z)

In [9]:
grid1 = simple_idw(x,y,z,xi,yi, power=3)
grid1 = grid1.reshape((ny, nx))

In [21]:
# Calcula e escreve em um arquivo os indices estatisticos calculados utilizando os dados interpolados dos pluviometros
index = verification.detcontscores.det_cont_fct(var_model, grid1)
mean_modelo = np.nanmean(var_model)
mean_obs = np.nanmean(grid1)
std_modelo = np.nanstd(var_model)
std_obs = np.nanstd(grid1)
f= open("Erros_PrecipAcum_Pluviometros.txt", "a")
f.write ("Mean_modelo_interp:"  + " " + str(mean_modelo) + "\n")
f.write ("Mean_obs_interp:" + " " + str(mean_obs) + "\n")
f.write ("Std_modelo_interp:" + " " + str(std_modelo) + "\n")
f.write ("Std_obs_interp:" + " " + str(std_obs) + "\n")
f.write ("BIAS_interp:" + " " + str(index['ME']) + "\n")
f.write ("RMSE_interp:" + " " + str(index['RMSE']) + "\n")
f.write ("Corr_spearman_interp:" + str(index['corr_s']) + "\n")

40

In [23]:
# Determina o ponto mais proximo na simulacao de cada estacao (pluviometro)
dfs = []
for i in range (len(dados)):
    ds = precip_model.sel(lat=dados['Lat'][i], lon=dados['Lon'][i], method="nearest")
#    print(ds)
    dfs.append(pd.concat(axis=0, ignore_index=True, objs=[
                         pd.DataFrame.from_dict({'Precip_obs': dados['Chuva'][i],
                                                 'Precip_brams': [ds.values]
                                                })
                            ]))
df_model = pd.concat(dfs, ignore_index=True)
df_model

,Precip_obs,Precip_brams
0,5.2,0.22273878753185272
1,0.0,6.134520053863525
2,0.0,0.8070045113563538
3,0.0,6.134520053863525
4,6.4,0.37081480026245117
...,...,...
433,0.0,-0.00030046049505472183
434,0.0,-0.00030046049505472183
435,0.0,-0.00014337152242660522
436,1.0,-0.0007658653194084764


In [25]:
# Calcula e escreve em um arquivo os indices estatisticos calculados utilizando os dados dos pluviometros (sem interpolacao)
index = verification.detcontscores.det_cont_fct(df_model['Precip_brams'].astype(float), df_model['Precip_obs'].astype(float))
mean_modelo = np.nanmean(df_model['Precip_brams'])
mean_obs = np.nanmean(df_model['Precip_obs'])
std_modelo = np.nanstd(df_model['Precip_brams'])
std_obs = np.nanstd(df_model['Precip_obs'])
f= open("Erros_PrecipAcum_Pluviometros.txt", "a")
f.write ("Mean_modelo:"  + " " + str(mean_modelo) + "\n")
f.write ("Mean_obs:" + " " + str(mean_obs) + "\n")
f.write ("Std_modelo:" + " " + str(std_modelo) + "\n")
f.write ("Std_obs:" + " " + str(std_obs) + "\n")
f.write ("BIAS:" + " " + str(index['ME']) + "\n")
f.write ("RMSE:" + " " + str(index['RMSE']) + "\n")
f.write ("Corr_spearman:" + str(index['corr_s']) + "\n")

34

In [27]:
# Calcula e escreve em um arquivo os indices estatisticos calculados utilizando os dados do MERGE
index = verification.detcontscores.det_cont_fct(var_model, var_merge)
mean_modelo = np.nanmean(var_model)
mean_obs = np.nanmean(var_merge)
std_modelo = np.nanstd(var_model)
std_obs = np.nanstd(var_merge)
f= open("Erros_PrecipAcum_MERGE.txt", "w")
f.write ("Mean_modelo:"  + " " + str(mean_modelo) + "\n")
f.write ("Mean_obs:" + " " + str(mean_obs) + "\n")
f.write ("Std_modelo:" + " " + str(std_modelo) + "\n")
f.write ("Std_obs:" + " " + str(std_obs) + "\n")
f.write ("BIAS:" + " " + str(index['ME']) + "\n")
f.write ("RMSE:" + " " + str(index['RMSE']) + "\n")
f.write ("Corr_spearman:" + str(index['corr_s']) + "\n")

34